In [1]:
import numpy as np

In [2]:
def read_log_file(filename):
    with open(filename) as fin:
        train_loss = []
        train_acc = []
        test_nll = []
        test_acc = []
        sparsity = []
        for line in fin:
            line_array = line.strip().split()
            if len(line_array) == 7 and line_array[0][0] not in ['-', 'e'] : 
                line_array = list(map(float, line_array))
                train_loss.append(line_array[2])
                train_acc.append(float(line_array[3]))
                test_nll.append(line_array[4])
                test_acc.append(float(line_array[5]))
            if len(line_array) == 3 and line_array[0][0] not in ['P', 't'] :
                sparsity.append(float(line_array[0]) / float(line_array[2]) * 100)
        if not len(sparsity):
            sparsity = np.ones(len(train_loss)) * 100
        sparsity = np.array(sparsity)
        sparsity = 100 - sparsity
    return list(map(np.array, [train_loss, train_acc, test_nll, test_acc, sparsity]))

In [3]:
def read_all_seeds(base_filename = "sgd.txt", seed_index_list = [1,2,3,4,5,6,7,8,9,10]):
    all_seeds_results = [[], [], [], [], []]
    for seed_index in seed_index_list:
        filename = str(seed_index) + "/" + base_filename
        results = read_log_file(filename)
        for i, result in enumerate(results):
            if not len(all_seeds_results[i]):
                all_seeds_results[i] = result
            else:
                all_seeds_results[i] = np.vstack((all_seeds_results[i], result))
    return np.array(all_seeds_results)

In [4]:
filenames_all = ['sgd.txt', 
                 'grda_c0.001_mu0.501.txt', 
                 'grda_c0.001_mu0.55.txt', 
                 'grda_c0.001_mu0.6.txt', 
                 'grda_c0.001_mu0.65.txt']

all_setting_results = []
for filename in filenames_all:
    all_seeds_results = read_all_seeds(base_filename = filename)
    all_setting_results.append(all_seeds_results)
all_setting_results = np.array(all_setting_results)

In [5]:
mean_vector = all_setting_results.mean(2)
sd_vector = all_setting_results.std(2)

In [6]:
print(",Epoch,1,25,50,75,100,200,")
measure_name = ["training_loss", 'tr_acc (%)', "testing_loss", "te_acc (%)", "sparsity (%)"]
setting_name = ['SGD', 'gRDA(0.501)', 'gRDA(0.55)', 'gRDA(0.6)', 'gRDA(0.65)']
decimals_val = 4
for measure_idx in [0,1,2,3,4]:
    for setting_idx in range(5):
        if setting_idx == 0:
            print(measure_name[measure_idx] + ',' + setting_name[setting_idx], end=",")
        else:
            print(',' + setting_name[setting_idx], end=",")
        for epoch_idx in [1,24,49,74,99,199]:
            print(str(np.around(mean_vector[setting_idx, measure_idx, epoch_idx], decimals=decimals_val)) + 
                    "(" + str(np.around(sd_vector[setting_idx, measure_idx, epoch_idx], decimals=decimals_val)) + ")", end=",")
        print()

,Epoch,1,25,50,75,100,200,
training_loss,SGD,2.9477(0.0037),0.3347(0.0038),0.0348(0.0044),0.0076(0.0023),0.0023(0.0028),0.0004(0.0),
,gRDA(0.501),2.9632(0.0052),0.3921(0.0031),0.0652(0.0047),0.033(0.0073),0.0233(0.0063),0.0006(0.0),
,gRDA(0.55),2.9752(0.0052),0.4167(0.0025),0.0758(0.005),0.0427(0.0055),0.0317(0.0077),0.0007(0.0),
,gRDA(0.6),2.9798(0.0038),0.4407(0.0042),0.0966(0.0041),0.0542(0.0039),0.0394(0.0075),0.0008(0.0),
,gRDA(0.65),2.9885(0.0062),0.4633(0.0035),0.1188(0.0046),0.0692(0.0053),0.0495(0.0038),0.0011(0.0),
tr_acc (%),SGD,25.7816(0.1029),89.2406(0.1343),99.071(0.1465),99.825(0.0637),99.9418(0.0723),99.9822(0.0038),
,gRDA(0.501),25.4118(0.0721),87.4652(0.141),98.069(0.1749),99.0446(0.256),99.327(0.2105),99.9792(0.0022),
,gRDA(0.55),25.188(0.1628),86.6598(0.0732),97.7116(0.1959),98.7498(0.2099),99.0674(0.2492),99.9794(0.0035),
,gRDA(0.6),25.092(0.0661),86.0122(0.1322),97.0026(0.1456),98.3896(0.1561),98.8524(0.253),99.977(0.0029),
,gRDA(0.65),24.9426(0.128),85.3238(0.106

In [7]:
import matplotlib.pyplot as plt
import numpy
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

fig, axs = plt.subplots(5, 1)
fig.set_figheight(10)
fig.set_figwidth(10)

window_size = 500
epoch_num = numpy.array(range(1,201))
setting_legends = []


axs_index = 0

################## training loss
axs_sub = axs[axs_index]
measure_idx = 0 # training loss
for setting_idx in range(5):
    color = next(axs_sub._get_lines.prop_cycler)['color']
    setting_legend, = axs_sub.plot(epoch_num, mean_vector[setting_idx,measure_idx,:], color=color)
    axs_sub.fill_between(epoch_num, 
                         mean_vector[setting_idx,measure_idx,:]+sd_vector[setting_idx,measure_idx,:], 
                         mean_vector[setting_idx,measure_idx,:]-sd_vector[setting_idx,measure_idx,:], 
                         color=color, alpha=0.2)
    setting_legends.append(setting_legend)
axs_sub.tick_params(axis='x', labelsize=14, bottom=False, top=False, labelbottom=False)
axs_sub.set_ylabel('train loss')
axs_sub.grid(True)


# zoom
# axins = zoomed_inset_axes(axs_sub, 3, loc="center right", bbox_to_anchor=(600,450), borderpad=3)
axins = inset_axes(axs_sub, width=5, height=0.8, loc="center right", bbox_to_anchor=(650,600), borderpad=3)
mark_inset(axs_sub, axins, loc1=1, loc2=3, fc="none", ec="0.5")
axins.set_xlim([150,200])
axins.set_ylim([0,0.003])

axs_sub = axins
for setting_idx in range(5):
    color = next(axs_sub._get_lines.prop_cycler)['color']
    axs_sub.plot(epoch_num, mean_vector[setting_idx,measure_idx,:],color=color)




################## training accuracy
axs_index += 1
axs_sub = axs[axs_index]
measure_idx = 1 # training accuracy
for setting_idx in range(5):
    color = next(axs_sub._get_lines.prop_cycler)['color']
    setting_legend, = axs_sub.plot(epoch_num, mean_vector[setting_idx,measure_idx,:], color=color)
    axs_sub.fill_between(epoch_num, 
                         mean_vector[setting_idx,measure_idx,:]+sd_vector[setting_idx,measure_idx,:], 
                         mean_vector[setting_idx,measure_idx,:]-sd_vector[setting_idx,measure_idx,:], 
                         color=color, alpha=0.2)
    setting_legends.append(setting_legend)
axs_sub.set_ylabel('train accuracy')
axs_sub.tick_params(axis='x', labelsize=14, bottom=False, top=False, labelbottom=False)
axs_sub.grid(True)

# zoom
# axins = zoomed_inset_axes(axs_sub, 3, loc="center right", bbox_to_anchor=(600,450), borderpad=3)
axins = inset_axes(axs_sub, width=5, height=0.8, loc="center right", bbox_to_anchor=(650,475), borderpad=3)
mark_inset(axs_sub, axins, loc1=2, loc2=4, fc="none", ec="0.5")
axins.set_xlim([150,200])
axins.set_ylim([99.94,100])

axs_sub = axins
for setting_idx in range(5):
    color = next(axs_sub._get_lines.prop_cycler)['color']
    axs_sub.plot(epoch_num, mean_vector[setting_idx,measure_idx,:],color=color)
# axs_sub.grid(True)






################## testing loss
axs_index += 1
axs_sub = axs[axs_index]
measure_idx = 2 # testing loss
for setting_idx in range(5):
    color = next(axs_sub._get_lines.prop_cycler)['color']
    # setting_legend, = axs_sub.plot(epoch_num, mean_vector[setting_idx,measure_idx,:], color=color)
    axs_sub.fill_between(epoch_num, 
                         mean_vector[setting_idx,measure_idx,:]+sd_vector[setting_idx,measure_idx,:], 
                         mean_vector[setting_idx,measure_idx,:]-sd_vector[setting_idx,measure_idx,:], 
                         color=color, alpha=0.5)
axs_sub.set_ylabel('test loss')
axs_sub.tick_params(axis='x', labelsize=14, bottom=False, top=False, labelbottom=False)
axs_sub.grid(True)

# zoom
# axins = zoomed_inset_axes(axs_sub, 20, loc="center right", bbox_to_anchor=(600,300), borderpad=3)
axins = inset_axes(axs_sub, width=5, height=0.7, loc="center right", bbox_to_anchor=(650,380), borderpad=3)
mark_inset(axs_sub, axins, loc1=1, loc2=3, fc="none", ec="0.5")
axins.set_xlim([150,200])
axins.set_ylim([1.35,1.55])

axs_sub = axins
for setting_idx in range(5):
    color = next(axs_sub._get_lines.prop_cycler)['color']
    axs_sub.plot(epoch_num,  mean_vector[setting_idx,measure_idx,:],color=color)
#     axs_sub.fill_between(epoch_num, 
#                          mean_vector[setting_idx,measure_idx,:]+sd_vector[setting_idx,measure_idx,:], 
#                          mean_vector[setting_idx,measure_idx,:]-sd_vector[setting_idx,measure_idx,:], 
#                          color=color, alpha=0.1)
plt.xticks(visible=True)
plt.yticks(visible=True)

################## testing accuracy
axs_index += 1
axs_sub = axs[axs_index]
measure_idx = 3 # testing accuracy
for setting_idx in range(5):
    color = next(axs_sub._get_lines.prop_cycler)['color']
    # setting_legend, = axs_sub.plot(epoch_num, mean_vector[setting_idx,measure_idx,:], color=color)
    axs_sub.fill_between(epoch_num, 
                         mean_vector[setting_idx,measure_idx,:]+sd_vector[setting_idx,measure_idx,:], 
                         mean_vector[setting_idx,measure_idx,:]-sd_vector[setting_idx,measure_idx,:], 
                         color=color, alpha=0.5)
axs_sub.set_ylabel('test accuracy')
axs_sub.tick_params(axis='x', labelsize=14, bottom=False, top=False, labelbottom=False)
axs_sub.grid(True)

# zoom
# axins = zoomed_inset_axes(axs_sub, 20, loc="center right", bbox_to_anchor=(600,300), borderpad=3)
axins = inset_axes(axs_sub, width=5, height=0.7, loc="center right", bbox_to_anchor=(650,245), borderpad=3)
mark_inset(axs_sub, axins, loc1=2, loc2=4, fc="none", ec="0.5")
axins.set_xlim([150,200])
axins.set_ylim([76.0,77.5])

axs_sub = axins
for setting_idx in range(5):
    color = next(axs_sub._get_lines.prop_cycler)['color']
    axs_sub.plot(epoch_num,  mean_vector[setting_idx,measure_idx,:],color=color)
#     axs_sub.fill_between(epoch_num, 
#                          mean_vector[setting_idx,measure_idx,:]+sd_vector[setting_idx,measure_idx,:], 
#                          mean_vector[setting_idx,measure_idx,:]-sd_vector[setting_idx,measure_idx,:], 
#                          color=color, alpha=0.1)
plt.xticks(visible=True)
plt.yticks(visible=True)



################## sparsity
axs_index += 1
axs_sub = axs[axs_index]
measure_idx = 4 # sparsity
for setting_idx in range(5):
    color = next(axs_sub._get_lines.prop_cycler)['color']
    setting_legend, = axs_sub.plot(epoch_num, mean_vector[setting_idx,measure_idx,:], color=color)
    axs_sub.fill_between(epoch_num, 
                         mean_vector[setting_idx,measure_idx,:]+sd_vector[setting_idx,measure_idx,:], 
                         mean_vector[setting_idx,measure_idx,:]-sd_vector[setting_idx,measure_idx,:], 
                         color=color, alpha=0.5)
axs_sub.set_xlabel('epoch')
axs_sub.set_ylabel('sparsity')
axs_sub.grid(True)


# axs[0].set_title('Compare SGD with gRDA (Data: CIFAR100, Model: WideResNet28x10, lr=0.1)')
axs[4].legend(setting_legends,[ "SGD (no other parameters)", "gRDA: c=0.001, mu=0.501", "gRDA: c=0.001, mu=0.55", "gRDA: c=0.001, mu=0.6", "gRDA: c=0.001, mu=0.65"], loc='lower right', ncol=2)
# plt.rcParams['font.weight'] = 'bold'
# plt.rcParams['font.weight'] = 'bold'
# plt.rcParams['axes.labelweight'] = 'bold'

# plt.tight_layout() 
# plt
plt.savefig("gRDA_SGD_cifar100_wrn28x10_10seeds_acc_loss.pdf")
